전자공시 OPEN DART 시스템

* [공시정보]-[공시검색]

* https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019001

In [3]:
import requests
import datetime
import pandas as pd
import os
import pickle

api_key = "d7ace37298ef7036447ceb35a4a09a683e963f4c"

corp_dict = {
    "orion" : "01238169"
}

def get_new_topic(corp_nm):

    # 날짜 계산
    today = datetime.datetime.today()
    end_de = today.strftime("%Y%m%d")
    today_pre_7 = today - datetime.timedelta(days=180)
    bgn_de = today_pre_7.strftime("%Y%m%d")
    corp_no = corp_dict.get(corp_nm)

    params = {
        "crtfc_key" : api_key,
        "bgn_de" : bgn_de,
        "end_de" : end_de,
        "corp_code" : corp_no
    }

    url = f"https://opendart.fss.or.kr/api/list.json"
    response = requests.get(url, params=params)
    rls = response.json()

    # 조회된 데이터가 없으면 종료
    if rls['status'] == '013':
        print('status 013, 조회된 데이터가 없습니다.')
        return 0

    df = pd.DataFrame(rls['list'])

    try:
        os.mkdir('./rcept_no_max')
    except:
        pass

    # rcept_no_max 확인
    if f"{corp_nm}_rcept_no_max.pkl" in os.listdir('./rcept_no_max'):
        with open(f'./rcept_no_max/{corp_nm}_rcept_no_max.pkl', 'rb') as f:
            orion_recept_no_max = pickle.load(f)

    else:
        # 파일이 없으면, 2번째로 큰 값을 가져온다
        orion_recept_no_max = list(df['rcept_no'].sort_values(ascending=False))[1]

    # max보다 큰 값이 있으면 신규 소식 있음
    out = df[df['rcept_no'] > orion_recept_no_max]

    if len(out) == 0:
        print('신규 소식이 없습니다.')
        return 0
    else:
        message = ""
        for o in out.iterrows():
            message = corp_nm + ", " + o[1]['rcept_dt'] + ', ' + message + o[1]['report_nm'] + ', ' + f"http://dart.fss.or.kr/dsaf001/main.do?rcpNo={url}" + "\n"

        with open(f'./rcept_no_max/{corp_nm}_rcept_no_max.pkl', 'wb') as f:
            orion_recept_no_max = df['rcept_no'].max()
            pickle.dump(orion_recept_no_max, f)

        return message

In [4]:
message = get_new_topic('orion')
print(message)

orion, 20221227, 임원ㆍ주요주주특정증권등소유상황보고서, http://dart.fss.or.kr/dsaf001/main.do?rcpNo=https://opendart.fss.or.kr/api/list.json

